In [2]:
%cd ~/REVIVAL2

/disk2/fli/REVIVAL2


In [3]:
%load_ext blackcellmagic

In [43]:
import os
import re
import pandas as pd
import numpy as np
from glob import glob
from scipy.stats import spearmanr

In [26]:
import glob
import os
import pandas as pd

# List of directory patterns with wildcards
zs_subdir_list = [
    "zs/esmif/*/score/ParLQ.csv",

]

# Columns to exclude from renaming
EXCLUDE_COLUMNS = {"var", "AAs", "selectivity", "fitness"}

# Convert patterns to match non-CSV files
csv_patterns = zs_subdir_list  # Keep as .csv

# Dictionary to store unique names for each path
unique_combos = {}

# Store all DataFrames for merging
combined_df = pd.DataFrame()

# Extract and name unique folder combinations
for pattern in csv_patterns:
    # Use glob to find matching CSV files
    csv_files = glob.glob(pattern, recursive=True)

    # Extract the unique portions of each path
    for csv_path in csv_files:
        path_parts = csv_path.split(os.sep)  # Split into directory components
        pattern_parts = pattern.split(os.sep)  # Reference split for wildcards
        
        # Extract the wildcard parts by comparing positions
        variable_parts = [part for part, ref in zip(path_parts, pattern_parts) if "*" in ref]
        
        # Generate a unique name for this combination
        unique_name = "-".join(variable_parts)

        # Store the unique name for reference
        unique_combos[csv_path] = unique_name  

        # Load CSV file
        df = pd.read_csv(csv_path)

        # Rename columns by appending unique_combo (unless in EXCLUDE_COLUMNS)
        df = df.rename(columns={col: f"{col}_{unique_name}" if col not in EXCLUDE_COLUMNS else col for col in df.columns})
        combine_col = [c for c in EXCLUDE_COLUMNS if c in df.columns]
        # Store processed DataFrame
        combined_df = pd.merge(combined_df, df, on=combine_col, how="outer") if not combined_df.empty else df

combined_df


,var,fitness,selectivity,esmif_score_apo,esmif_score_docked
0,F89A,0.000000,0.000000,-1.017301,-0.987254
1,F89C,4.215013,0.174286,-1.027136,-1.005826
2,F89D,1.585337,0.025801,-1.000953,-0.985437
3,F89E,2.487884,0.090939,-0.999779,-0.990325
4,F89G,0.000000,0.000000,-1.043611,-1.018762
...,...,...,...,...,...
485,Y57R,5.120007,0.266769,-1.017647,-1.006563
486,Y57S,14.675230,0.422914,-1.020287,-1.004204
487,Y57T,13.030966,0.373729,-1.020497,-0.998909
488,Y57V,12.362720,0.310562,-1.022741,-1.014909


In [91]:
EXCLUDE_COLUMNS = {"var", "AAs", "selectivity", "fitness"}
# Regex pattern to ignore columns with "_0_", "_1_" inside or ending with "_0" or "_1"
IGNORE_PATTERN = re.compile(r".*_(\d+)$")


def parse_zs_dir_csv(pattern):

    # Dictionary to store unique names for each path
    # unique_combos = {}

    # Store all DataFrames for merging
    combined_df = pd.DataFrame()

    # Use glob to find matching CSV files
    csv_files = glob(pattern, recursive=True)
    print(csv_files)

    # Extract the unique portions of each path
    for csv_path in csv_files:

        # Replace `/` with `-` to create a unique identifier
        unique_name = "-".join(csv_path.split("/")[2:-1])

        print(unique_name)

        # Store the unique name for reference
        # unique_combos[csv_path] = unique_name  

        # Load CSV file
        df = pd.read_csv(csv_path)
        print(df.columns)
        # Remove columns that match IGNORE_PATTERN
        df = df.loc[:, ~df.columns.str.match(IGNORE_PATTERN)]
        print(df.columns)

        # Rename columns by appending unique_combo (unless in EXCLUDE_COLUMNS)
        df = df.rename(columns={col: f"{col}_{unique_name}" if col not in EXCLUDE_COLUMNS else col for col in df.columns})
        print(df.columns)
        combine_col = [c for c in EXCLUDE_COLUMNS if c in df.columns and c in combined_df.columns]
        print(combine_col)
        # Store processed DataFrame
        combined_df = pd.merge(combined_df, df, on=combine_col, how="outer") if not combined_df.empty else df
    return combined_df

In [83]:
parse_zs_dir_csv("zs/esmif/*/score/ParLQ.csv")

['zs/esmif/apo/score/ParLQ.csv', 'zs/esmif/docked/score/ParLQ.csv']
Index(['var', 'fitness', 'selectivity', 'esmif_score'], dtype='object')
Index(['var', 'fitness', 'selectivity', 'esmif_score'], dtype='object')
Index(['var', 'fitness', 'selectivity', 'esmif_score_apo-score'], dtype='object')
[]
Index(['var', 'fitness', 'selectivity', 'esmif_score'], dtype='object')
Index(['var', 'fitness', 'selectivity', 'esmif_score'], dtype='object')
Index(['var', 'fitness', 'selectivity', 'esmif_score_docked-score'], dtype='object')
['var', 'fitness', 'selectivity']


,var,fitness,selectivity,esmif_score_apo-score,esmif_score_docked-score
0,F89A,0.000000,0.000000,-1.017301,-0.987254
1,F89C,4.215013,0.174286,-1.027136,-1.005826
2,F89D,1.585337,0.025801,-1.000953,-0.985437
3,F89E,2.487884,0.090939,-0.999779,-0.990325
4,F89G,0.000000,0.000000,-1.043611,-1.018762
...,...,...,...,...,...
485,Y57R,5.120007,0.266769,-1.017647,-1.006563
486,Y57S,14.675230,0.422914,-1.020287,-1.004204
487,Y57T,13.030966,0.373729,-1.020497,-0.998909
488,Y57V,12.362720,0.310562,-1.022741,-1.014909


In [84]:
parse_zs_dir_csv("zs/coves/*/output/100_processed/ParLQ.csv")

# "triad/score/*/*.csv",
# "af3/score_*/*.csv",
# "chai/score_*/*.csv",
# "bonddist/*/*/*.csv",
# "hydro/**/*.csv",
# "plip/*/*/*.csv",
# "vina/*/*/*.csv",

['zs/coves/apo_clean/output/100_processed/ParLQ.csv', 'zs/coves/sub_clean/output/100_processed/ParLQ.csv']
Index(['var', 'fitness', 'selectivity', 'coves_score'], dtype='object')
Index(['var', 'fitness', 'selectivity', 'coves_score'], dtype='object')
Index(['var', 'fitness', 'selectivity',
       'coves_score_apo_clean-output-100_processed'],
      dtype='object')
[]
Index(['var', 'fitness', 'selectivity', 'coves_score'], dtype='object')
Index(['var', 'fitness', 'selectivity', 'coves_score'], dtype='object')
Index(['var', 'fitness', 'selectivity',
       'coves_score_sub_clean-output-100_processed'],
      dtype='object')
['var', 'fitness', 'selectivity']


,var,fitness,selectivity,coves_score_apo_clean-output-100_processed,coves_score_sub_clean-output-100_processed
0,F89A,0.000000,0.000000,-1231.904945,-1180.578237
1,F89C,4.215013,0.174286,-1226.842148,-1174.173820
2,F89D,1.585337,0.025801,-1270.660360,-1210.035472
3,F89E,2.487884,0.090939,-1252.240264,-1190.921901
4,F89G,0.000000,0.000000,-1263.729350,-1215.518094
...,...,...,...,...,...
485,Y57R,5.120007,0.266769,-1241.907143,-1197.408870
486,Y57S,14.675230,0.422914,-1224.385972,-1173.638233
487,Y57T,13.030966,0.373729,-1209.388659,-1155.280380
488,Y57V,12.362720,0.310562,-1221.757058,-1188.097328


In [85]:
parse_zs_dir_csv("zs/triad/score/*/ParLQ.csv")


# "af3/score_*/*.csv",
# "chai/score_*/*.csv",
# "bonddist/*/*/*.csv",
# "hydro/**/*.csv",
# "plip/*/*/*.csv",
# "vina/*/*/*.csv",

['zs/triad/score/frompdb-cleanup/ParLQ.csv', 'zs/triad/score/docked/ParLQ.csv']
Index(['AAs', 'fitness', 'Triad_score', 'Triad_rank'], dtype='object')
Index(['AAs', 'fitness', 'Triad_score', 'Triad_rank'], dtype='object')
Index(['AAs', 'fitness', 'Triad_score_score-frompdb-cleanup',
       'Triad_rank_score-frompdb-cleanup'],
      dtype='object')
[]
Index(['AAs', 'fitness', 'Triad_score', 'Triad_rank'], dtype='object')
Index(['AAs', 'fitness', 'Triad_score', 'Triad_rank'], dtype='object')
Index(['AAs', 'fitness', 'Triad_score_score-docked',
       'Triad_rank_score-docked'],
      dtype='object')
['fitness', 'AAs']


,AAs,fitness,Triad_score_score-frompdb-cleanup,Triad_rank_score-frompdb-cleanup,Triad_score_score-docked,Triad_rank_score-docked
0,QYRGL,0.000000,-635.78808,281,-629.85419,287
1,SYLQF,0.000000,-641.78646,151,-638.89200,101
2,WYLQA,0.000000,-651.74783,42,-645.35036,46
3,WYLQG,0.000000,-648.65452,75,-642.80420,72
4,WYLQS,0.000000,-650.49888,57,-644.90568,52
...,...,...,...,...,...,...
485,FYFNY,81.914062,-648.01330,80,-641.65513,83
486,MGFDY,83.627237,-636.72509,265,-630.57751,271
487,MKFNY,85.802944,-638.44746,231,-632.89298,213
488,FKMDY,86.706154,-639.64153,200,-634.49908,178


In [86]:
parse_zs_dir_csv("zs/af3/score_*/ParLQ.csv")


# "chai/score_*/*.csv",
# "bonddist/*/*/*.csv",
# "hydro/**/*.csv",
# "plip/*/*/*.csv",
# "vina/*/*/*.csv",

['zs/af3/score_seperate/ParLQ.csv', 'zs/af3/score_joint/ParLQ.csv']
Index(['var', 'ranking_score_0', 'mean_site_score_0', 'ptm_0', 'iptm_0',
       'fraction_disordered_0', 'chain_ptm_A_0', 'chain_iptm_A_0',
       'chain_pae_min_AA_0', 'chain_pae_min_AB_0',
       ...
       'chain_iptm_CA_avg', 'chain_iptm_CA_std', 'chain_pae_min_CB_avg',
       'chain_pae_min_CB_std', 'chain_iptm_CB_avg', 'chain_iptm_CB_std',
       'chain_pae_min_CC_avg', 'chain_pae_min_CC_std', 'mean_site_score_avg',
       'mean_site_score_std'],
      dtype='object', length=209)
Index(['var', 'ranking_score_agg', 'mean_site_score_agg', 'ptm_agg',
       'iptm_agg', 'fraction_disordered_agg', 'chain_ptm_A_agg',
       'chain_iptm_A_agg', 'chain_pae_min_AA_agg', 'chain_pae_min_AB_agg',
       'chain_iptm_AB_agg', 'chain_pae_min_AC_agg', 'chain_iptm_AC_agg',
       'chain_ptm_B_agg', 'chain_iptm_B_agg', 'chain_pae_min_BA_agg',
       'chain_iptm_BA_agg', 'chain_pae_min_BB_agg', 'chain_pae_min_BC_agg',
       'chain

,var,ranking_score_agg_score_seperate,mean_site_score_agg_score_seperate,ptm_agg_score_seperate,iptm_agg_score_seperate,fraction_disordered_agg_score_seperate,chain_ptm_A_agg_score_seperate,chain_iptm_A_agg_score_seperate,chain_pae_min_AA_agg_score_seperate,chain_pae_min_AB_agg_score_seperate,...,chain_iptm_B_avg_score_joint,chain_iptm_B_std_score_joint,chain_pae_min_BA_avg_score_joint,chain_pae_min_BA_std_score_joint,chain_iptm_BA_avg_score_joint,chain_iptm_BA_std_score_joint,chain_pae_min_BB_avg_score_joint,chain_pae_min_BB_std_score_joint,mean_site_score_avg_score_joint,mean_site_score_std_score_joint
0,F89A,0.89,98.214,0.93,0.88,0.0,0.94,0.84,0.76,2.39,...,0.870000,0.018257,1.383333,0.063421,0.870000,0.018257,0.76,1.110223e-16,98.070667,0.125362
1,F89C,0.88,98.178,0.93,0.87,0.0,0.94,0.84,0.76,2.33,...,0.860000,0.018257,1.413333,0.061824,0.860000,0.018257,0.76,1.110223e-16,98.009000,0.176427
2,F89D,0.89,98.292,0.93,0.88,0.0,0.94,0.86,0.76,2.20,...,0.883333,0.004714,1.331667,0.030777,0.883333,0.004714,0.76,1.110223e-16,98.247333,0.092597
3,F89E,0.89,98.190,0.93,0.88,0.0,0.94,0.85,0.76,2.23,...,0.845000,0.018028,1.488333,0.060116,0.845000,0.018028,0.76,1.110223e-16,97.852333,0.117912
4,F89G,0.88,98.014,0.92,0.87,0.0,0.94,0.84,0.76,2.37,...,0.848333,0.017717,1.460000,0.073937,0.848333,0.017717,0.76,1.110223e-16,98.049667,0.129663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Y57R,0.89,97.920,0.92,0.88,0.0,0.93,0.85,0.76,2.35,...,0.850000,0.018257,1.393333,0.090860,0.850000,0.018257,0.76,1.110223e-16,97.932667,0.063725
486,Y57S,0.89,97.756,0.92,0.88,0.0,0.93,0.85,0.76,2.17,...,0.836667,0.015986,1.376667,0.073182,0.836667,0.015986,0.76,1.110223e-16,97.594667,0.132553
487,Y57T,0.89,97.818,0.92,0.88,0.0,0.93,0.84,0.76,2.32,...,0.833333,0.016997,1.465000,0.087702,0.833333,0.016997,0.76,1.110223e-16,97.554000,0.126196
488,Y57V,0.89,97.676,0.92,0.89,0.0,0.93,0.86,0.76,2.18,...,0.828333,0.013437,1.510000,0.054160,0.828333,0.013437,0.76,1.110223e-16,97.705000,0.075831


In [87]:
parse_zs_dir_csv("zs/chai/score_*/ParLQ.csv")


# "bonddist/*/*/*.csv",
# "hydro/**/*.csv",
# "plip/*/*/*.csv",
# "vina/*/*/*.csv",

['zs/chai/score_seperate/ParLQ.csv', 'zs/chai/score_joint/ParLQ.csv']


Index(['var', 'aggregate_score_0', 'ptm_0', 'iptm_0', 'mean_site_score_0',
       'chain_ptm_A_0', 'chain_iptm_AB_0', 'chain_iptm_AC_0', 'chain_ptm_B_0',
       'chain_iptm_BA_0', 'chain_iptm_BC_0', 'chain_ptm_C_0',
       'chain_iptm_CA_0', 'chain_iptm_CB_0', 'aggregate_score_1', 'ptm_1',
       'iptm_1', 'mean_site_score_1', 'chain_ptm_A_1', 'chain_iptm_AB_1',
       'chain_iptm_AC_1', 'chain_ptm_B_1', 'chain_iptm_BA_1',
       'chain_iptm_BC_1', 'chain_ptm_C_1', 'chain_iptm_CA_1',
       'chain_iptm_CB_1', 'aggregate_score_2', 'ptm_2', 'iptm_2',
       'mean_site_score_2', 'chain_ptm_A_2', 'chain_iptm_AB_2',
       'chain_iptm_AC_2', 'chain_ptm_B_2', 'chain_iptm_BA_2',
       'chain_iptm_BC_2', 'chain_ptm_C_2', 'chain_iptm_CA_2',
       'chain_iptm_CB_2', 'aggregate_score_3', 'ptm_3', 'iptm_3',
       'mean_site_score_3', 'chain_ptm_A_3', 'chain_iptm_AB_3',
       'chain_iptm_AC_3', 'chain_ptm_B_3', 'chain_iptm_BA_3',
       'chain_iptm_BC_3', 'chain_ptm_C_3', 'chain_iptm_CA_3',
   

,var,aggregate_score_avg_score_seperate,aggregate_score_std_score_seperate,ptm_avg_score_seperate,ptm_std_score_seperate,iptm_avg_score_seperate,iptm_std_score_seperate,chain_ptm_A_avg_score_seperate,chain_ptm_A_std_score_seperate,chain_iptm_AB_avg_score_seperate,...,chain_ptm_A_avg_score_joint,chain_ptm_A_std_score_joint,chain_iptm_AB_avg_score_joint,chain_iptm_AB_std_score_joint,chain_ptm_B_avg_score_joint,chain_ptm_B_std_score_joint,chain_iptm_BA_avg_score_joint,chain_iptm_BA_std_score_joint,mean_site_score_avg_score_joint,mean_site_score_std_score_joint
0,F89A,0.705190,0.017906,0.897363,0.001393,0.657147,0.022047,0.913913,0.000350,0.016978,...,0.919751,0.000260,0.762178,0.003182,0.857185,0.001428,0.797767,0.001585,90.300512,0.156640
1,F89C,0.799650,0.002277,0.907747,0.000559,0.772626,0.002843,0.916048,0.000735,0.017415,...,0.916070,0.000191,0.647810,0.027356,0.834686,0.004097,0.630759,0.056209,90.157355,0.129022
2,F89D,0.819678,0.000672,0.914046,0.000416,0.796086,0.000801,0.921672,0.000302,0.017729,...,0.918250,0.000242,0.745774,0.003780,0.854749,0.003692,0.793397,0.001348,89.920569,0.048991
3,F89E,0.812735,0.001550,0.911442,0.000403,0.788058,0.001906,0.918154,0.000247,0.019175,...,0.918015,0.000341,0.742058,0.005514,0.848011,0.000522,0.780573,0.002115,88.261092,0.286351
4,F89G,0.722215,0.072053,0.898435,0.009259,0.678159,0.087755,0.912799,0.000631,0.017497,...,0.918981,0.000206,0.755649,0.001450,0.856564,0.001508,0.800585,0.001379,90.389787,0.052051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Y57R,0.741306,0.002963,0.912663,0.001431,0.698467,0.003598,0.920507,0.001861,0.024597,...,0.911997,0.001375,0.657893,0.004329,0.829170,0.002729,0.668573,0.006794,87.443215,0.143139
486,Y57S,0.767907,0.054769,0.911290,0.007295,0.732061,0.066642,0.917632,0.000437,0.025269,...,0.907975,0.000222,0.669712,0.012881,0.828412,0.001846,0.694652,0.024018,90.600463,0.114003
487,Y57T,0.758230,0.058695,0.905306,0.008119,0.721461,0.071345,0.914449,0.001063,0.019802,...,0.905726,0.000269,0.637946,0.014563,0.823791,0.001043,0.663389,0.023573,89.463082,0.067780
488,Y57V,0.724707,0.026419,0.901455,0.002999,0.680519,0.032300,0.915002,0.001550,0.016647,...,0.908650,0.001284,0.636188,0.007736,0.824527,0.001222,0.665191,0.010621,90.081935,0.148373


In [88]:
parse_zs_dir_csv("zs/bonddist/*/*/ParLQ.csv")

# "hydro/**/*.csv",
# "plip/*/*/*.csv",
# "vina/*/*/*.csv",

['zs/bonddist/af3/struct_joint/ParLQ.csv', 'zs/bonddist/af3/struct_seperate/ParLQ.csv', 'zs/bonddist/chai/struct_joint/ParLQ.csv', 'zs/bonddist/chai/struct_seperate/ParLQ.csv']
Index(['var', 'fitness', 'selectivity', '0:C-C_1_0', '0:C-C_1_1', '0:C-C_1_2',
       '0:C-C_1_3', '0:C-C_1_4', '0:C-C_1_agg', '0:C-C_1_avg', '0:C-C_1_std',
       '1:C-C_2_0', '1:C-C_2_1', '1:C-C_2_2', '1:C-C_2_3', '1:C-C_2_4',
       '1:C-C_2_agg', '1:C-C_2_avg', '1:C-C_2_std'],
      dtype='object')
Index(['var', 'fitness', 'selectivity', '0:C-C_1_agg', '0:C-C_1_avg',
       '0:C-C_1_std', '1:C-C_2_agg', '1:C-C_2_avg', '1:C-C_2_std'],
      dtype='object')
Index(['var', 'fitness', 'selectivity', '0:C-C_1_agg_af3-struct_joint',
       '0:C-C_1_avg_af3-struct_joint', '0:C-C_1_std_af3-struct_joint',
       '1:C-C_2_agg_af3-struct_joint', '1:C-C_2_avg_af3-struct_joint',
       '1:C-C_2_std_af3-struct_joint'],
      dtype='object')
[]
Index(['var', 'fitness', 'selectivity', '0:C-C_1_0', '0:C-C_1_1', '0:C-C_1_2',
 

,var,fitness,selectivity,0:C-C_1_agg_af3-struct_joint,0:C-C_1_avg_af3-struct_joint,0:C-C_1_std_af3-struct_joint,1:C-C_2_agg_af3-struct_joint,1:C-C_2_avg_af3-struct_joint,1:C-C_2_std_af3-struct_joint,0:C-C_1_agg_af3-struct_seperate,...,1:C-C_2_avg_af3-struct_seperate,1:C-C_2_std_af3-struct_seperate,0:C-C_1_avg_chai-struct_joint,0:C-C_1_std_chai-struct_joint,1:C-C_2_avg_chai-struct_joint,1:C-C_2_std_chai-struct_joint,0:C-C_1_avg_chai-struct_seperate,0:C-C_1_std_chai-struct_seperate,1:C-C_2_avg_chai-struct_seperate,1:C-C_2_std_chai-struct_seperate
0,F89A,0.000000,0.000000,8.385017,7.649181,0.566010,8.208241,7.650717,0.431493,6.764923,...,4.557356,1.295375,5.254085,1.031610,4.673459,0.812476,5.277256,1.194438,4.756904,0.744705
1,F89C,4.215013,0.174286,8.499862,7.872733,0.514597,8.367809,7.807237,0.427585,7.079286,...,4.434683,1.396272,5.326030,0.475751,4.894778,0.726133,5.783040,1.538332,5.067429,1.399067
2,F89D,1.585337,0.025801,8.193742,7.705960,0.359133,8.266835,7.809575,0.258398,7.581095,...,5.643827,1.881452,5.777965,1.227881,4.955388,1.232168,5.526261,0.945287,4.951802,0.464629
3,F89E,2.487884,0.090939,6.183751,7.449763,0.793042,5.987115,7.228517,1.135682,3.125487,...,5.618915,1.427626,5.311500,1.240649,5.094703,0.599681,8.458034,1.659309,7.419762,1.418605
4,F89G,0.000000,0.000000,6.417363,5.715385,1.037487,5.211043,4.894315,0.391759,4.712509,...,5.105754,1.025357,4.723418,0.462230,4.385077,0.707222,5.059125,0.981903,4.830502,0.506502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Y57R,5.120007,0.266769,7.241956,6.172563,1.224373,6.924783,5.955826,0.574801,3.579368,...,4.909423,0.291843,5.967641,0.905997,5.133711,0.512175,8.236410,0.911732,7.013311,0.927521
486,Y57S,14.675230,0.422914,7.465871,6.900308,0.353062,6.348056,6.090416,0.607765,3.661495,...,5.052237,0.643262,7.351674,1.532408,6.349958,1.349010,8.941164,0.110906,7.848767,0.268051
487,Y57T,13.030966,0.373729,7.303405,6.358527,0.746006,6.134398,5.871052,0.297781,2.516898,...,4.757138,1.570023,6.807295,1.140668,6.033952,1.088663,8.693885,0.244441,7.684144,0.335457
488,Y57V,12.362720,0.310562,7.322856,6.569565,0.644795,6.033467,5.936445,0.229274,2.882311,...,4.774474,1.544276,5.973978,0.429486,4.885157,0.361777,8.850004,0.313228,7.901783,0.274858


In [89]:
parse_zs_dir_csv("zs/hydro/**/ParLQ.csv")


# "plip/*/*/*.csv",
# "vina/*/*/*.csv",

['zs/hydro/af3/struct_joint/ParLQ.csv', 'zs/hydro/af3/struct_separate/ParLQ.csv', 'zs/hydro/frompdb/ParLQ.csv', 'zs/hydro/chai/struct_joint/ParLQ.csv', 'zs/hydro/chai/struct_separate/ParLQ.csv']
Index(['var', 'fitness', 'pocket-plip-sasa_0', 'pocket-plip-sasa_1',
       'pocket-plip-sasa_2', 'pocket-plip-sasa_3', 'pocket-plip-sasa_4',
       'pocket-plip-sasa_avg', 'pocket-plip-kd_0', 'pocket-plip-kd_1',
       ...
       'pocket-subcentroid-hw_agg', 'pocket-subcentroid-ec_agg',
       'pocket-subcentroid-sasa-theor_agg', 'pocket-subcentroid-sasa-emp_agg',
       'pocket-subcofcentroid-kd_agg', 'pocket-subcofcentroid-hw_agg',
       'pocket-subcofcentroid-ec_agg', 'pocket-subcofcentroid-sasa-theor_agg',
       'pocket-subcofcentroid-sasa-emp_agg', 'rep'],
      dtype='object', length=137)
Index(['var', 'fitness', 'pocket-plip-sasa_avg', 'pocket-plip-kd_avg',
       'pocket-plip-hw_avg', 'pocket-plip-ec_avg',
       'pocket-plip-sasa-theor_avg', 'pocket-plip-sasa-emp_avg',
       'pocke

,var,fitness,pocket-plip-sasa_avg_af3-struct_joint,pocket-plip-kd_avg_af3-struct_joint,pocket-plip-hw_avg_af3-struct_joint,pocket-plip-ec_avg_af3-struct_joint,pocket-plip-sasa-theor_avg_af3-struct_joint,pocket-plip-sasa-emp_avg_af3-struct_joint,pocket-subcentroid-sasa_avg_af3-struct_joint,pocket-subcentroid-kd_avg_af3-struct_joint,...,pocket-subcentroid-hw_avg_chai-struct_separate,pocket-subcentroid-ec_avg_chai-struct_separate,pocket-subcentroid-sasa-theor_avg_chai-struct_separate,pocket-subcentroid-sasa-emp_avg_chai-struct_separate,pocket-subcofcentroid-sasa_avg_chai-struct_separate,pocket-subcofcentroid-kd_avg_chai-struct_separate,pocket-subcofcentroid-hw_avg_chai-struct_separate,pocket-subcofcentroid-ec_avg_chai-struct_separate,pocket-subcofcentroid-sasa-theor_avg_chai-struct_separate,pocket-subcofcentroid-sasa-emp_avg_chai-struct_separate
0,F89A,0.000000,5527.254764,0.171074,-0.636066,0.238888,199.928839,191.223766,2211.774933,0.784105,...,-1.163957,0.543775,203.221846,194.058676,2964.140052,0.632753,-1.187991,0.376375,222.877530,212.445723
1,F89C,4.215013,5515.186456,0.189231,-0.651548,0.237537,200.430672,191.578578,2226.943175,0.773421,...,-0.880915,0.443444,199.304510,189.463072,2975.055773,0.813905,-1.229905,0.415048,221.485714,210.532381
2,F89D,1.585337,5522.603848,0.142636,-0.615917,0.229942,200.697501,192.025641,2205.565615,0.464678,...,-0.968783,0.465382,204.920217,195.739022,2937.477215,0.216162,-0.965639,0.261968,227.147068,216.869784
3,F89E,2.487884,5527.651974,0.096139,-0.582308,0.219521,201.094998,192.404529,2251.093700,0.371526,...,-0.571971,0.304922,205.496618,196.095343,2976.195731,0.653404,-1.104341,0.399240,226.800201,216.190226
4,F89G,0.000000,5470.712889,0.155076,-0.629267,0.240495,200.196294,191.516028,2158.249577,0.864327,...,-0.994386,0.374513,212.327987,203.122807,2944.997715,0.283429,-1.045095,0.311419,218.623810,208.369048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Y57R,5.120007,5573.954156,0.143504,-0.558090,0.208303,202.363751,193.660794,2168.943317,0.908317,...,-0.653364,0.284202,206.643366,197.363333,3006.587181,0.916351,-1.337827,0.466278,228.090000,217.347243
486,Y57S,14.675230,5574.493893,0.128788,-0.559630,0.223488,200.439399,191.696994,2109.607127,1.152026,...,-0.937059,0.419824,204.311029,194.610294,3016.130247,0.922030,-1.342677,0.455407,227.998797,217.033835
487,Y57T,13.030966,5582.884855,0.124643,-0.563201,0.223491,201.157446,192.448545,2093.647089,1.245041,...,-0.892361,0.397597,206.350000,196.788889,2989.323563,0.948862,-1.412110,0.480368,228.403935,217.551604
488,Y57V,12.362720,5571.342557,0.194345,-0.584545,0.240853,200.873332,192.169585,2112.369925,1.563150,...,-0.912761,0.445639,204.867484,195.472222,3049.711030,1.115376,-1.389338,0.488489,229.377368,218.578271


In [93]:
parse_zs_dir_csv("zs/plip/*/*/ParLQ.csv")

# "vina/*/*/*.csv",

['zs/plip/af3/score_seperate/ParLQ.csv', 'zs/plip/af3/score_joint/ParLQ.csv', 'zs/plip/chai/score_seperate/ParLQ.csv', 'zs/plip/chai/score_joint/ParLQ.csv']
af3-score_seperate
Index(['var', 'fitness', 'selectivity', 'plip_naive_score_0',
       'plip_naive_score_1', 'plip_naive_score_2', 'plip_naive_score_3',
       'plip_naive_score_4', 'plip_naive_score_avg', 'num_hydrophobic_0',
       ...
       'sum_metal_complex_agg', 'num_water_bridge_agg', 'sum_water_bridge_agg',
       'num_pi_stack_agg', 'sum_pi_stack_agg', 'num_pi_cation_agg',
       'sum_pi_cation_agg', 'num_halogen_bond_agg', 'sum_halogen_bond_agg',
       'num_interactions_agg'],
      dtype='object', length=138)
Index(['var', 'fitness', 'selectivity', 'plip_naive_score_avg',
       'num_hydrophobic_avg', 'sum_hydrophobic_avg', 'num_hydrogen_bond_avg',
       'sum_hydrogen_bond_avg', 'num_salt_bridge_avg', 'sum_salt_bridge_avg',
       'num_metal_complex_avg', 'sum_metal_complex_avg',
       'num_water_bridge_avg', 'sum_w

,var,fitness,selectivity,plip_naive_score_avg_af3-score_seperate,num_hydrophobic_avg_af3-score_seperate,sum_hydrophobic_avg_af3-score_seperate,num_hydrogen_bond_avg_af3-score_seperate,sum_hydrogen_bond_avg_af3-score_seperate,num_salt_bridge_avg_af3-score_seperate,sum_salt_bridge_avg_af3-score_seperate,...,sum_metal_complex_avg_chai-score_joint,num_water_bridge_avg_chai-score_joint,sum_water_bridge_avg_chai-score_joint,num_pi_stack_avg_chai-score_joint,sum_pi_stack_avg_chai-score_joint,num_pi_cation_avg_chai-score_joint,sum_pi_cation_avg_chai-score_joint,num_halogen_bond_avg_chai-score_joint,sum_halogen_bond_avg_chai-score_joint,num_interactions_avg_chai-score_joint
0,F89A,0.000000,0.000000,-14.193434,25.0,-1.60650,3.2,-0.103934,2.0,-0.448,...,-11.00,0.0,0.0,0.2,-0.530,0.0,0.0,0.0,0.0,35.6
1,F89C,4.215013,0.174286,-13.804462,21.8,-1.49362,3.0,-0.117842,2.0,-0.488,...,-10.71,0.0,0.0,0.4,-0.290,0.0,0.0,0.0,0.0,37.4
2,F89D,1.585337,0.025801,-14.983547,24.4,-1.51538,3.4,-0.091167,2.0,-0.592,...,-11.35,0.0,0.0,0.6,-0.720,0.0,0.0,0.0,0.0,37.4
3,F89E,2.487884,0.090939,-14.481587,25.8,-1.59154,3.8,-0.092047,2.0,-0.508,...,-11.07,0.0,0.0,1.0,-1.120,0.0,0.0,0.0,0.0,38.6
4,F89G,0.000000,0.000000,-13.808629,24.4,-1.43718,3.6,-0.111449,2.0,-0.560,...,-11.07,0.0,0.0,0.2,-0.085,0.0,0.0,0.0,0.0,35.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Y57R,5.120007,0.266769,-13.452170,23.8,-1.49396,3.0,-0.123210,2.0,0.000,...,-11.08,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,38.4
486,Y57S,14.675230,0.422914,-13.219198,23.6,-1.45384,3.2,-0.125358,2.0,0.000,...,-10.52,0.0,0.0,0.2,-0.215,0.0,0.0,0.0,0.0,39.2
487,Y57T,13.030966,0.373729,-13.412842,22.4,-1.45248,2.6,-0.135362,2.0,0.000,...,-10.56,0.0,0.0,0.4,-0.140,0.0,0.0,0.0,0.0,39.4
488,Y57V,12.362720,0.310562,-13.626958,23.2,-1.49498,3.0,-0.126978,2.0,0.000,...,-10.57,0.0,0.0,0.6,-0.400,0.0,0.0,0.0,0.0,41.8


In [108]:
parse_zs_dir_csv("zs/vina/*/*/*/ParLQ.csv")

['zs/vina/af3/score_seperate/substrate_cofactor-docked/ParLQ.csv', 'zs/vina/af3/score_seperate/substrate_cofactor-score_only/ParLQ.csv', 'zs/vina/af3/score_seperate/substrate+carbene_cofactor-score_only/ParLQ.csv', 'zs/vina/af3/score_joint/substrate_cofactor-docked/ParLQ.csv', 'zs/vina/af3/score_joint/substrate_cofactor-score_only/ParLQ.csv', 'zs/vina/apo/score/substrate_cofactor-docked/ParLQ.csv', 'zs/vina/apo/score/all_cofactor-docked/ParLQ.csv', 'zs/vina/chai/score_seperate/substrate_cofactor-docked/ParLQ.csv', 'zs/vina/chai/score_seperate/substrate+carbene_cofactor-docked/ParLQ.csv', 'zs/vina/chai/score_seperate/substrate_cofactor-score_only/ParLQ.csv', 'zs/vina/chai/score_seperate/substrate+carbene_cofactor-score_only/ParLQ.csv', 'zs/vina/chai/score_joint/substrate_cofactor-docked/ParLQ.csv', 'zs/vina/chai/score_joint/substrate+carbene_cofactor-docked/ParLQ.csv', 'zs/vina/chai/score_joint/substrate_cofactor-score_only/ParLQ.csv', 'zs/vina/chai/score_joint/substrate+carbene_cofacto

,var,fitness,selectivity,vina_af3-score_seperate-substrate_cofactor-docked,vina_min_af3-score_seperate-substrate_cofactor-docked,vina_max_af3-score_seperate-substrate_cofactor-docked,vina_af3-score_seperate-substrate_cofactor-score_only,vina_min_af3-score_seperate-substrate_cofactor-score_only,vina_max_af3-score_seperate-substrate_cofactor-score_only,vina_af3-score_seperate-substrate+carbene_cofactor-score_only,...,vina_max_chai-score_joint-substrate_cofactor-docked,vina_chai-score_joint-substrate+carbene_cofactor-docked,vina_min_chai-score_joint-substrate+carbene_cofactor-docked,vina_max_chai-score_joint-substrate+carbene_cofactor-docked,vina_chai-score_joint-substrate_cofactor-score_only,vina_min_chai-score_joint-substrate_cofactor-score_only,vina_max_chai-score_joint-substrate_cofactor-score_only,vina_chai-score_joint-substrate+carbene_cofactor-score_only,vina_min_chai-score_joint-substrate+carbene_cofactor-score_only,vina_max_chai-score_joint-substrate+carbene_cofactor-score_only
0,F89A,0.000000,0.000000,-5.4452,-6.899,-4.263,-2.9254,-5.377,0.539,-1.4438,...,-3.848,-8.5214,-9.536,-6.633,3.9152,-4.927,14.588,5.7420,-2.362,15.466
1,F89C,4.215013,0.174286,-5.5710,-6.906,-4.361,-3.0118,-5.364,0.542,-1.6322,...,-5.217,-9.2284,-9.626,-8.817,-3.3796,-4.345,-1.802,-1.6322,-2.802,0.180
2,F89D,1.585337,0.025801,-4.7180,-5.939,-4.109,-1.7456,-4.962,0.426,-1.0924,...,-3.897,-8.7944,-9.675,-7.589,6.3478,-3.642,15.122,8.3438,-0.670,15.730
3,F89E,2.487884,0.090939,-4.7534,-5.624,-4.157,-1.9726,-4.257,-0.316,-0.5612,...,-3.580,-8.1112,-9.529,-6.378,1.3006,-1.239,3.965,4.8678,2.837,7.448
4,F89G,0.000000,0.000000,-5.9610,-6.888,-4.948,-3.4910,-5.330,-2.242,-2.5230,...,-4.556,-8.5296,-9.476,-7.296,-4.0322,-5.764,-2.030,-1.4432,-4.318,0.829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Y57R,5.120007,0.266769,-6.3470,-6.668,-6.022,-4.9526,-5.405,-4.408,-3.8836,...,-5.521,-9.0042,-9.732,-8.037,-3.0130,-4.280,-0.959,-0.7226,-2.627,2.231
486,Y57S,14.675230,0.422914,-6.1242,-6.453,-5.800,-4.9894,-5.479,-4.031,-3.7226,...,-4.482,-8.8588,-10.230,-7.530,-3.2056,-4.327,-2.253,-0.9008,-2.081,0.845
487,Y57T,13.030966,0.373729,-5.9920,-6.338,-5.403,-3.0486,-4.773,0.372,-2.0688,...,-5.810,-8.5750,-9.706,-7.002,-4.1596,-5.182,-3.531,-1.3030,-3.116,1.201
488,Y57V,12.362720,0.310562,-5.6004,-5.973,-5.131,-3.4438,-4.348,-1.474,-2.5236,...,-5.942,-8.7920,-9.415,-8.137,-1.0820,-5.278,2.115,1.5004,-3.556,5.044


In [111]:
zs_subdir_list = [
            "ev/*.csv",
            "esm/output/*.csv",
            "flowsite/scores/pocket_def_residues_model2-substrate-cofactor/*.csv",
            "ligandmpnn/scores/autoregressive_20/*.csv",
            "vol/*.csv",  #
            "esmif/*/score/*.csv",
            "coves/*/output/100_processed/*.csv",
            "triad/score/*/*.csv",
            "af3/score_*/*.csv",  # need to look at apo
            "chai/score_*/*.csv",  # need to look at apo
            "bonddist/*/*/*.csv",
            "hydro/**/*.csv",
            "plip/*/*/*.csv",
            "vina/*/*/*/*.csv",  # add af3 chai sub dock from smiles
        ]

In [112]:
zs_dir = "zs"

In [113]:
[os.path.join(zs_dir, f) for f in zs_subdir_list if f.count("*") == 1]

['zs/ev/*.csv',
 'zs/esm/output/*.csv',
 'zs/flowsite/scores/pocket_def_residues_model2-substrate-cofactor/*.csv',
 'zs/ligandmpnn/scores/autoregressive_20/*.csv',
 'zs/vol/*.csv']

In [115]:
[os.path.join(zs_dir, f) for f in zs_subdir_list if f.count("*") > 1]

['zs/esmif/*/score/*.csv',
 'zs/coves/*/output/100_processed/*.csv',
 'zs/triad/score/*/*.csv',
 'zs/af3/score_*/*.csv',
 'zs/chai/score_*/*.csv',
 'zs/bonddist/*/*/*.csv',
 'zs/hydro/**/*.csv',
 'zs/plip/*/*/*.csv',
 'zs/vina/*/*/*/*.csv']